In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import datetime
plt.style.use('ggplot')

In [33]:
tests = pd.read_csv("test_table.csv",index_col='user_id')
users = pd.read_csv("user_table.csv",index_col='user_id')

tests = tests.join(users)

tests['date'] = pd.to_datetime(tests.date)
tests['signup_date'] = pd.to_datetime(tests.signup_date)

In [34]:
tests.sample(5)

,date,browser,test,pages_visited,signup_date
user_id,,,,,
2349223,2015-08-23,Firefox,1,4,2015-03-11
269040,2015-08-26,IE,1,5,2015-01-09
5095551,2015-08-02,Safari,0,9,2015-05-31
564678,2015-08-01,Chrome,0,9,2015-01-17
7574647,2015-08-07,IE,1,2,2015-08-07


In [35]:
def run_ttest(df):
    pv_in_test = df.loc[tests.test == 1, 'pages_visited']
    test_mean = pv_in_test.mean()
    
    pv_in_ctrl = df.loc[tests.test == 0, 'pages_visited']
    ctrl_mean = pv_in_ctrl.mean()
    
    result = ss.ttest_ind(pv_in_ctrl, pv_in_test, equal_var=False)
    conclusion = 'Significant' if result.pvalue < 0.05 else 'Not Significant'
    
    return pd.Series({'n_test':pv_in_test.shape[0],
                      'n_ctrl': pv_in_ctrl.shape[0],
                      'mean_test': test_mean,
                      'mean_ctrl': ctrl_mean,
                      'test-ctrl': test_mean - ctrl_mean,
                      'pvalue':result.pvalue,
                      'conclusion':conclusion})

In [36]:
import scipy.stats as ss
run_ttest(tests)

conclusion    Not Significant
mean_ctrl             4.60839
mean_test             4.59969
n_ctrl                  49846
n_test                  50154
pvalue               0.577452
test-ctrl         -0.00870091
dtype: object

In [37]:
tests.groupby('browser').apply(run_ttest)

,conclusion,mean_ctrl,mean_test,n_ctrl,n_test,pvalue,test-ctrl
browser,,,,,,,
Chrome,Significant,4.613341,4.690680,21453,21974,9.434084e-04,0.077339
Firefox,Significant,4.600164,4.714259,10972,10786,5.817199e-04,0.114095
IE,Significant,4.598478,4.685985,10906,10974,7.829509e-03,0.087507
Opera,Significant,4.546438,0.000000,1109,1018,2.252939e-321,-4.546438
Safari,Not Significant,4.638180,4.692336,5406,5402,2.411738e-01,0.054156


In [38]:
min(tests.date)

Timestamp('2015-08-01 00:00:00')

divide customers into 2 group, signup date = test date and the other.

In [39]:
tests.loc[tests.date == tests.signup_date,'signup_equal_test_date'] = 1

In [40]:
tests.signup_equal_test_date = tests.signup_equal_test_date.fillna(0)

In [41]:
tests.signup_equal_test_date.value_counts()

0.0    79999
1.0    20001
Name: signup_equal_test_date, dtype: int64

In [42]:
tests.groupby('signup_equal_test_date').apply(run_ttest)

,conclusion,mean_ctrl,mean_test,n_ctrl,n_test,pvalue,test-ctrl
signup_equal_test_date,,,,,,,
0.0,Not Significant,4.603284,4.622379,39890,40109,0.261837,0.019095
1.0,Significant,4.628867,4.509109,9956,10045,0.001742,-0.119758


In [44]:
tests.groupby(['browser','signup_equal_test_date']).apply(run_ttest)

conclusion  mean_ctrl  mean_test  n_ctrl  \
browser signup_equal_test_date                                                  
Chrome  0.0                         Significant   4.607945   4.701512   17092   
        1.0                     Not Significant   4.634488   4.648011    4361   
Firefox 0.0                         Significant   4.590590   4.757306    8842   
        1.0                     Not Significant   4.639906   4.539220    2130   
IE      0.0                         Significant   4.590576   4.721494    8744   
        1.0                     Not Significant   4.630435   4.543964    2162   
Opera   0.0                         Significant   4.594564   0.000000     883   
        1.0                         Significant   4.358407   0.000000     226   
Safari  0.0                     Not Significant   4.638254   4.720973    4329   
        1.0                     Not Significant   4.637883   4.578657    1077   

                                n_test         pvalue  test-ctrl  
browser signup_equal_test_date                                    
Chrome  0.0                      17525   2.290889e-04   0.093567  
        1.0                       4449   8.149175e-01   0.013523  
Firefox 0.0                       8657   3.692901e-06   0.166716  
        1.0                       2129   2.210706e-01  -0.100686  
IE      0.0                       8779   2.669847e-04   0.130918  
        1.0                       2195   2.808421e-01  -0.086471  
Opera   0.0                        833  7.204927e-255  -4.594564  
        1.0                        185   1.222949e-68  -4.358407  
Safari  0.0                       4315   1.000829e-01   0.082720  
        1.0                       1087   6.015241e-01  -0.059226

* something wrong with Safari browser, we need to double check UI or code
* New user do not impacted by the new feature, because they do not have recommendation yet. (cold start) We can create new feature for new users such as popular people instead of friends recommentdation
* Opera has a bug on new feature, the page view is 0
